<a href="https://colab.research.google.com/github/ZJCHEN0109/AIMoedl_SmartMed/blob/main/MedCareIsAAOrBB(word2vec).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#套件安裝

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import glob
import numpy as np
import os
import tensorflow as tf
import jieba.analyse
import jieba
import jieba.posseg as pseg

In [ ]:
from urllib.request import urlretrieve
url="https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url,"dict.txt.big")
jieba.set_dictionary("dict.txt.big")
## 使用者自訂字典
# jieba.load_userdict("dict.news.txt")

# 資料來源

基本參數(預留BERT使用)

In [ ]:
FOLDER_PATH = '/content/drive/MyDrive/AI工程師/AI進階深度學習/05-Transformer/BERT'

In [ ]:
# df = pd.read_csv(os.path.join(FOLDER_PATH, 'Gensim-SmartMedCare.csv'), header=None)
train_df = pd.read_csv(os.path.join(FOLDER_PATH, 'Gensim-SmartMedCare.csv'))
test_df = pd.read_csv(os.path.join(FOLDER_PATH, 'Gensim-SmartMedCare.csv'))

In [ ]:
train_df

,dfcontent,sentiment
0,"服用adarone,宜避免或小心服用葡萄柚汁(請錯開二小時服用)",不了解處方用藥的應注意事項
1,"Topamax剝半或磨粉後會釋出苦味，因病人使用鼻胃管,所以不致影響。此藥會有體重降低情形，...",不了解處方用藥的應注意事項
2,服用bromazepam須注意本品可能會引發順行性健忘。使用較高治療劑量時可能發生順行性健忘...,不了解處方用藥的應注意事項
3,Amlobin、Celecolen會造成水腫現象，墊高抬腿防止減輕症狀,不了解處方用藥的應注意事項
4,使用Crestor需注意肌肉痠痛的情形\r,不了解處方用藥的應注意事項
...,...,...
635,Plavix與Xarelto的交互作用,藥品-藥品交互作用
636,Omeprazole與Eltroxin的交互作用,藥品-藥品交互作用
637,Neurontin與Baclofen交互作用,藥品-藥品交互作用
638,Plavix和Piracetam交互作用,藥品-藥品交互作用


In [ ]:
test_df

,dfcontent,sentiment
0,"服用adarone,宜避免或小心服用葡萄柚汁(請錯開二小時服用)",不了解處方用藥的應注意事項
1,"Topamax剝半或磨粉後會釋出苦味，因病人使用鼻胃管,所以不致影響。此藥會有體重降低情形，...",不了解處方用藥的應注意事項
2,服用bromazepam須注意本品可能會引發順行性健忘。使用較高治療劑量時可能發生順行性健忘...,不了解處方用藥的應注意事項
3,Amlobin、Celecolen會造成水腫現象，墊高抬腿防止減輕症狀,不了解處方用藥的應注意事項
4,使用Crestor需注意肌肉痠痛的情形\r,不了解處方用藥的應注意事項
...,...,...
635,Plavix與Xarelto的交互作用,藥品-藥品交互作用
636,Omeprazole與Eltroxin的交互作用,藥品-藥品交互作用
637,Neurontin與Baclofen交互作用,藥品-藥品交互作用
638,Plavix和Piracetam交互作用,藥品-藥品交互作用


#文字轉換成數字

In [ ]:
##統計唯一出現次數最多詞彙
# train_df["sentiment"].unique()
test_df["sentiment"].unique()

array(['不了解處方用藥的應注意事項', '不瞭解劑型操作方法', '用來治療另一藥可避免的副作用', '吃藥時間過於複雜',
       '向原處方醫師確認', '因交互作用造成劑量降低', '在正常劑量下，產生不期望的藥理反應', '有未治療的急性狀況或疾病',
       '有治療禁忌', '此藥沒有適應症存在)', '自我照護技巧/生活型態不佳', '使用OTC藥/保健食品/中草藥知識不正確',
       '使用不安全藥品', '治療期間過長，藥量累積', '建議生化、血液或療效監測', '建議改變用藥間隔', '建議改變治療期限',
       '建議改變劑型', '建議改變劑量', '建議更改給藥時間/用藥方法', '建議更改藥品數量', '建議停用某藥',
       '建議換用另一種藥品', '建議開始用某藥', '重覆用藥(同一種藥或同一藥理分類)', '病人肝腎功能不佳',
       '缺乏可支持的檢驗數據', '從前用此處方藥物治療失敗', '幾種用藥給藥時間太複雜', '無法吞下或給藥',
       '對病人不安全(如疾病危險因子、懷孕、哺乳、幼兒、老人)', '劑量過低，或血中濃度不夠', '劑量過高',
       '應給予預防性藥物治療', '還有更有效、安全、方便或便宜的藥', '藥品劑型不適當', '藥品-藥品交互作用'],
      dtype=object)

In [ ]:
##屬於AA碼描述
names_pos = ["重覆用藥(同一種藥或同一藥理分類)","藥品劑型不適當","劑量過高","藥品-藥品交互作用","病人肝腎功能不佳","在正常劑量下，產生不期望的藥理反應","有未治療的急性狀況或疾病","其他"]
names_pos_ans = ["AA22","AA31","AA51","AA61","AA55","AA65","AA11","AA00"]
##屬於BB碼描述
names_neg = ["建議停用某藥","建議換用另一種藥品","建議改變劑量","建議改變用藥間隔","向原處方醫師確認","不了解處方用藥的應注意事項","建議生化、血液或療效監測","其他"]
names_neg_ans = ["BB12","BB13","BB14","BB16","BB22","AA84","BB23","BB00"]

newdic = {}
# for i in range(len(names)):
#     newdic[names[i]] = i
# 融合
newdicAA = {names_pos[i]:i for i in range(len(names_pos))}
newdicBB = {names_neg[i]:i for i in range(len(names_neg))}
# newdicAA
newdicBB

{'不了解處方用藥的應注意事項': 5,
 '其他': 7,
 '向原處方醫師確認': 4,
 '建議停用某藥': 0,
 '建議換用另一種藥品': 1,
 '建議改變劑量': 2,
 '建議改變用藥間隔': 3,
 '建議生化、血液或療效監測': 6}

In [ ]:
#記得傳到任何ML/DL函式庫 要把所有東西轉成數字
# series.replace(字典)
# test_df["sentiment"].replace({'重覆用藥(同一種藥或同一藥理分類)': 0, '藥品劑型不適當': 1, '劑量過高': 2, '藥品-藥品交互作用': 3, '建議停用某藥': 4, '建議改變劑量': 5, '建議改變用藥間隔': 6})
# test_df["sentiment"].value_counts() #回傳獨特值和次數
# train_df["sentiment"].unique()
##屬於AA碼描述
namedic = {names_pos[i]:i for i in range(len(names_pos))}
idxname = {i:names_pos[i] for i in range(len(names_pos))}
print(namedic)
print(idxname)
namedic_ans = {names_pos_ans[i]:i for i in range(len(names_pos_ans))}
idxname_ans = {i:names_pos_ans[i] for i in range(len(names_pos_ans))}
print(namedic_ans)
print(idxname_ans)

{'重覆用藥(同一種藥或同一藥理分類)': 0, '藥品劑型不適當': 1, '劑量過高': 2, '藥品-藥品交互作用': 3, '病人肝腎功能不佳': 4, '在正常劑量下，產生不期望的藥理反應': 5, '有未治療的急性狀況或疾病': 6, '其他': 7}
{0: '重覆用藥(同一種藥或同一藥理分類)', 1: '藥品劑型不適當', 2: '劑量過高', 3: '藥品-藥品交互作用', 4: '病人肝腎功能不佳', 5: '在正常劑量下，產生不期望的藥理反應', 6: '有未治療的急性狀況或疾病', 7: '其他'}
{'AA22': 0, 'AA31': 1, 'AA51': 2, 'AA61': 3, 'AA55': 4, 'AA65': 5, 'AA11': 6, 'AA00': 7}
{0: 'AA22', 1: 'AA31', 2: 'AA51', 3: 'AA61', 4: 'AA55', 5: 'AA65', 6: 'AA11', 7: 'AA00'}


In [ ]:
##屬於BB碼描述
name2dic = {names_neg[i]:i for i in range(len(names_neg))}
idx2name = {i:names_neg[i] for i in range(len(names_neg))}
print(name2dic)
print(idx2name)
name2dic_ans = {names_neg_ans[i]:i for i in range(len(names_neg_ans))}
idx2name_ans = {i:names_neg_ans[i] for i in range(len(names_neg_ans))}
print(name2dic_ans)
print(idx2name_ans)

{'建議停用某藥': 0, '建議換用另一種藥品': 1, '建議改變劑量': 2, '建議改變用藥間隔': 3, '向原處方醫師確認': 4, '不了解處方用藥的應注意事項': 5, '建議生化、血液或療效監測': 6, '其他': 7}
{0: '建議停用某藥', 1: '建議換用另一種藥品', 2: '建議改變劑量', 3: '建議改變用藥間隔', 4: '向原處方醫師確認', 5: '不了解處方用藥的應注意事項', 6: '建議生化、血液或療效監測', 7: '其他'}
{'BB12': 0, 'BB13': 1, 'BB14': 2, 'BB16': 3, 'BB22': 4, 'AA84': 5, 'BB23': 6, 'BB00': 7}
{0: 'BB12', 1: 'BB13', 2: 'BB14', 3: 'BB16', 4: 'BB22', 5: 'AA84', 6: 'BB23', 7: 'BB00'}


In [ ]:
# train_df = train_df["sentiment"].replace(namedic)
# test_df = test_df["sentiment"].replace(name2dic)
# train_df = np.array(train_df)
# test_df = np.array(test_df)

In [ ]:
##屬於AA碼描述
def namefilter(sentname):
    if sentname in namedic.keys():
        values = namedic.get(sentname)
        # ABCode = idxname_ans.get(values)
        return values
        # return ABCode
    else:
        return 7
        # return 'AA00'
        #pass
train_df["sentiment"] = train_df["sentiment"].apply(namefilter)

In [ ]:
train_df[150:200]

,dfcontent,sentiment
150,"鐵劑和其他藥品併用,需間隔使用",7
151,服用Iwell的注意事項\r,7
152,"Amantadine(Dopadine)與Memantine,Switane併用,會增加,即...",5
153,Omezol須注意當懷疑是胃潰瘍時，應先確認其並非惡性腫瘤，因本治療可能會減輕其症狀，並延誤...,5
154,"病人有腹瀉情形,Metformin與Exelon,MgO皆會有腹瀉的副作用,尤其是metfo...",5
155,"Lipitor若與含有niacin的B-COMPLEX併用,需留意肌肉疼痛及橫紋肌溶解的肌肉...",5
156,"Plavix與Nicametate併用,抗血小板作用會增強,須小心不正常出血。\r",5
157,服用Livalo須注意肌肉疼痛的副作用。,5
158,注意有無皮膚紅疹的問題。\r,5
159,樂多糖漿經腸道分解後，產生類似單糖成分，但在治療便秘的劑量上(20ml)，不致影響血糖值波動...,5


In [ ]:
# ##屬於BB碼描述
# def namefilter(sentname):
#     if sentname in name2dic:
#         values = name2dic.get(sentname)
#         # ABCode = idx2name_ans.get(values)
#         return values
#         #return ABCode
#     else:
#         return 7
#         # return 'BB00'
#         # pass
# test_df["sentiment"] = test_df["sentiment"].apply(namefilter)

In [ ]:
##屬於BB碼描述(因沒有Test測試資料使用Train取代)
def namefilter(sentname):
    if sentname in namedic.keys():
        values = namedic.get(sentname)
        # ABCode = idxname_ans.get(values)
        return values
        # return ABCode
    else:
        return 7
        # return 'AA00'
        #pass
train_df["sentiment"] = train_df["sentiment"].apply(namefilter)

In [ ]:
test_df[100:200]

,dfcontent,sentiment
100,降血糖藥建議可以改用複方+單方,7
101,"sennoside與dulcolax有複方藥品conslife,可考慮更改為conslife...",7
102,"血糖用藥可考慮使用複方產品,減少用藥品項及頻次",7
103,sennoside與dulcolax各別使用上是否造成服藥順從性的不便,7
104,Omezol須注意當懷疑是胃潰瘍時，應先確認其並非惡性腫瘤，因本治療可能會減輕其症狀，並延誤...,4
...,...,...
195,因4/8報告中有TG太高，達到248，建議回報醫師，並在近期回測,7
196,建議可以測量骨質疏鬆T-core,7
197,建議至胃腸科檢查糞便潛血，若無胃腸道疾病，可以服用營養品\r\n建議持續追蹤血鉀，補充含鉀食物。,7
198,住民近期心跳偏慢，雖有使用concor25mg，但影響有限，建議反應給該科別醫生,7


#將所有中文內容轉換為數字序列
#Step1 將所有內容作斷詞
#Step2 建立空字典
#Step3 查看所有內容，每出現一個字典裡沒有的詞彙，將該詞彙指定一個索引數放入字典
#Step4 利用建立好的字典將所有內容裏頭包含的詞彙轉換成數字

#建立使用者詞典(Token)
#方法一 利用Jieba製作Token，斷詞/分詞jiaba.cut()

In [ ]:
cols = ["dfcontent","sentiment"]
train_df = train_df.loc[:,cols]
train_df.head(3)
# train_df.index

,dfcontent,sentiment
0,"服用adarone,宜避免或小心服用葡萄柚汁(請錯開二小時服用)",7
1,"Topamax剝半或磨粉後會釋出苦味，因病人使用鼻胃管,所以不致影響。此藥會有體重降低情形，...",7
2,服用bromazepam須注意本品可能會引發順行性健忘。使用較高治療劑量時可能發生順行性健忘...,7


In [ ]:
cols = ["dfcontent","sentiment"]
test_df = test_df.loc[:,cols]
test_df.head(3)
# train_df.index

,dfcontent,sentiment
0,"服用adarone,宜避免或小心服用葡萄柚汁(請錯開二小時服用)",5
1,"Topamax剝半或磨粉後會釋出苦味，因病人使用鼻胃管,所以不致影響。此藥會有體重降低情形，...",5
2,服用bromazepam須注意本品可能會引發順行性健忘。使用較高治療劑量時可能發生順行性健忘...,5


In [ ]:
# import jieba.posseg as pseg
def jieba_tokenizer(text):
    words = pseg.cut(text)
    return ' '.join([word for word, flag in words if flag != 'x'])

In [ ]:
train_df.loc[:,"dfcontent"].apply(jieba_tokenizer)

0                   服用 adarone 宜 避免 或 小心 服用 葡萄柚 汁 請 二 服用
1      Topamax 剝半 或 磨粉 後 會 苦味 因 病人 使用 鼻胃 管 所以 不致 此 藥會...
2      服用 bromazepam 須 注意 本品 可能 會 性 健忘 使用 較 高 治療 時 可能...
3                           Amlobin Celecolen 會 造成 抬腿 防止
4                              使用 Crestor 需注意 肌肉 痠痛 的 情形
                             ...                        
635                              Plavix 與 Xarelto 的 交互作用
636                         Omeprazole 與 Eltroxin 的 交互作用
637                            Neurontin 與 Baclofen 交互作用
638                              Plavix 和 Piracetam 交互作用
639                                           此 藥 有 交互作用
Name: dfcontent, Length: 640, dtype: object

In [ ]:
test_df.loc[:,"dfcontent"].apply(jieba_tokenizer)

0                   服用 adarone 宜 避免 或 小心 服用 葡萄柚 汁 請 二 服用
1      Topamax 剝半 或 磨粉 後 會 苦味 因 病人 使用 鼻胃 管 所以 不致 此 藥會...
2      服用 bromazepam 須 注意 本品 可能 會 性 健忘 使用 較 高 治療 時 可能...
3                           Amlobin Celecolen 會 造成 抬腿 防止
4                              使用 Crestor 需注意 肌肉 痠痛 的 情形
                             ...                        
635                              Plavix 與 Xarelto 的 交互作用
636                         Omeprazole 與 Eltroxin 的 交互作用
637                            Neurontin 與 Baclofen 交互作用
638                              Plavix 和 Piracetam 交互作用
639                                           此 藥 有 交互作用
Name: dfcontent, Length: 640, dtype: object

In [ ]:
train_df["dfcontent"] = train_df.loc[:,"dfcontent"].apply(jieba_tokenizer)
test_df["dfcontent"] = test_df.loc[:,"dfcontent"].apply(jieba_tokenizer)

In [ ]:
# train_df["dfcontent"]
test_df["dfcontent"]

0                   服用 adarone 宜 避免 或 小心 服用 葡萄柚 汁 請 二 服用
1      Topamax 剝半 或 磨粉 後 會 苦味 因 病人 使用 鼻胃 管 所以 不致 此 藥會...
2      服用 bromazepam 須 注意 本品 可能 會 性 健忘 使用 較 高 治療 時 可能...
3                           Amlobin Celecolen 會 造成 抬腿 防止
4                              使用 Crestor 需注意 肌肉 痠痛 的 情形
                             ...                        
635                              Plavix 與 Xarelto 的 交互作用
636                         Omeprazole 與 Eltroxin 的 交互作用
637                            Neurontin 與 Baclofen 交互作用
638                              Plavix 和 Piracetam 交互作用
639                                           此 藥 有 交互作用
Name: dfcontent, Length: 640, dtype: object

#將AA和BB內容(content)匯集建立字典即為稱作語料庫(TextCorpus),所有的文本數據

In [ ]:
corpus_x1 = train_df.dfcontent
corpus_x2 = test_df.dfcontent
corpus = pd.concat([
    corpus_x1, corpus_x2])
corpus.shape

(1280,)

In [ ]:
pd.DataFrame(corpus.iloc[:5],columns=['dfcontent'])

,dfcontent
0,服用 adarone 宜 避免 或 小心 服用 葡萄柚 汁 請 二 服用
1,Topamax 剝半 或 磨粉 後 會 苦味 因 病人 使用 鼻胃 管 所以 不致 此 藥會...
2,服用 bromazepam 須 注意 本品 可能 會 性 健忘 使用 較 高 治療 時 可能...
3,Amlobin Celecolen 會 造成 抬腿 防止
4,使用 Crestor 需注意 肌肉 痠痛 的 情形


In [ ]:
#利用語料庫建立字典
# 1. Tokenize: I love you -> 1 2 3
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(corpus)

In [ ]:
x1_train = tok.texts_to_sequences(corpus_x1)
x2_train = tok.texts_to_sequences(corpus_x2)

In [ ]:
len(x1_train)

640

In [ ]:
x1_train[:1]

[[5, 403, 284, 46, 33, 285, 5, 201, 202, 21, 514, 5]]

In [ ]:
for seq in x1_train[:1]:
  print([tok.index_word[idx] for idx in seq])

['服用', 'adarone', '宜', '避免', '或', '小心', '服用', '葡萄柚', '汁', '請', '二', '服用']


In [ ]:
##計算最長詞彙長度
max_seq_len = max([len(seq) for seq in x2_train])
max_seq_len

62

#方法二 利用TensorFlow製作Token

# 將文本轉換成數字序列

In [ ]:
#Step1. Tokenize: I love you -> 1 2 3
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["dfcontent"])

In [ ]:
#產生字典
tok.word_index
# tok.index_word
x_train_seq = tok.texts_to_sequences(train_df["dfcontent"])
x_test_seq = tok.texts_to_sequences(test_df["dfcontent"])

In [ ]:
# len(x_train_seq)
# len(x_test_seq)
# x_train_seq[:1]
# x_test_seq[:1]
for seq in x_train_seq[:1]:
  print([tok.index_word[idx] for idx in seq])

['服用', 'adarone', '宜', '避免', '或', '小心', '服用', '葡萄柚', '汁', '請', '二', '服用']


In [ ]:
##將索引數字對應回本來的詞彙
for seq in x_train_seq[:2]:
    print([tok.index_word[idx] for idx in seq])

['服用', 'adarone', '宜', '避免', '或', '小心', '服用', '葡萄柚', '汁', '請', '二', '服用']
['topamax', '剝半', '或', '磨粉', '後', '會', '苦味', '因', '病人', '使用', '鼻胃', '管', '所以', '不致', '此', '藥會', '有', '降低', '情形', '避免', '與', 'ketorolac', 'naproxen', '等', '併', '用', '因為', '可能', '會', '造成', '加重', '若需', '時須']


In [ ]:
##查看最長詞彙長度
# max_seq_len = max([len(seq) for seq in x_train_seq])
max_seq_len = max([len(seq) for seq in x_train_seq])
max_seq_len

62

In [ ]:
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,5,403.0,284.0,46.0,33.0,285.0,5.0,201.0,202.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,515,286.0,33.0,102.0,37.0,4.0,516.0,38.0,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,404.0,23.0,12.0,288.0,17.0,4.0,57.0,289.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,291,405.0,4.0,60.0,525.0,292.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,406.0,124.0,65.0,131.0,1.0,19.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,62,3.0,508.0,1.0,22.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
636,195,3.0,130.0,1.0,22.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
637,144,3.0,479.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638,62,26.0,80.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# word2vec 轉成高維空間向量

In [ ]:
#Step2. padding: 把所有序列變成一樣長 把所有序列截長補短 預處理
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=62)
x_test_pad = pad_sequences(x_test_seq, maxlen=62)

In [ ]:
# pd.DataFrame(x_train_pad)
pd.DataFrame(x_test_pad)

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,0,0,0,0,0,0,0,0,0,0,...,284,46,33,285,5,201,202,21,514,5
1,0,0,0,0,0,0,0,0,0,0,...,71,9,10,51,17,4,60,520,521,522
2,0,0,0,0,0,0,0,0,0,0,...,57,289,28,523,524,290,34,45,290,34
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,291,405,4,60,525,292
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,6,406,124,65,131,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,62,3,508,1,22
636,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,195,3,130,1,22
637,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,144,3,479,22
638,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,62,26,80,22


##將正解做 One-hot Encoding 分類欄位 label 要進行從文本到數字的轉換

In [ ]:
# train_df.sentiment[:-1] ##train_df
# print(type(train.sentiment[:-1]))

0      7
1      7
2      7
3      7
4      7
      ..
634    3
635    3
636    3
637    3
638    3
Name: sentiment, Length: 639, dtype: int64

In [ ]:
# import numpy as np 

# ##定義 train_df 每一個分類對應到的索引數字
# label_to_index = {
#     "AA22":0,
#     "AA31":1,
#     "AA51":2,
#     "AA61":3,
#     "AA55":4,
#     "AA65":5,
#     "AA11":6,
#     "AA00":7
# }

# # 將分類標籤對應到剛定義的數字
# y_train = train_df.sentiment.apply(
#     lambda x: label_to_index[x])

# y_train = np.asarray(y_train).astype('float32')

# print(len(y_train[:-1]))
# y_train[:3]

639


array([7., 7., 7.], dtype=float32)

In [ ]:
# from keras.utils import np_utils
# y_train = np_utils.to_categorical(y_train, num_classes=8, dtype='float32')


In [ ]:
# y_train[100:200]

array(['AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00',
       'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00',
       'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00',
       'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00',
       'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00',
       'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00', 'AA00',
       'AA00', 'AA00', 'AA00', 'AA00', 'AA65', 'AA65', 'AA65', 'AA65',
       'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65',
       'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65',
       'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65',
       'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65', 'AA65',
       'AA65', 'AA65', 'AA65', 'AA65', 'AA11', 'AA11', 'AA11', 'AA11',
       'AA11', 'AA11', 'AA11', 'AA11'], dtype=object)

In [ ]:
##轉換為空間向量
# train_df = np.array(train_df)
# test_df = np.array(test_df)

#整個訓練資料集（Training Set）切成 2 個部分90%/10% 避免Overfitting產生

# 建立空模型

In [ ]:
from threading import active_count
#Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GlobalAveragePooling1D,Dense

layers=[
  Embedding(3001,100,mask_zero=True,input_length=62),
  GlobalAveragePooling1D(),
  Dense(8,activation="softmax")
]
model=Sequential(layers)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 62, 100)           300100    
                                                                 
 global_average_pooling1d_1   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 8)                 808       
                                                                 
Total params: 300,908
Trainable params: 300,908
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])

In [ ]:
y_train.shape
# y_test

(640,)

In [ ]:
# batch_size: 你看多少筆再進行一次梯度下降(5[圖片大/訓練資料少]-200[圖片小/訓練資料多])
# epochs: 全體資料(60000) 你要看幾輪(我們什麼時候要停的關鍵)
# validation_split: 測試什麼時候停下來要用沒看過的資料(val loss穩定的時候就停, 不要過擬合)
# (60000 * 0.9) / 200 -> 270
# verbose: 0(quiet) 1(full) 2(no progress bar)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callbacks = [
    ModelCheckpoint("imdb.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad,
     y_train,
     batch_size=200,
     epochs=50,
     validation_split=0.1,
     verbose=2,
     callbacks=callbacks)

Epoch 1/50
3/3 - 1s - loss: 2.0736 - accuracy: 0.2743 - val_loss: 2.0865 - val_accuracy: 0.0156 - 836ms/epoch - 279ms/step
Epoch 2/50
3/3 - 0s - loss: 2.0498 - accuracy: 0.6302 - val_loss: 2.0876 - val_accuracy: 0.0156 - 51ms/epoch - 17ms/step
Epoch 3/50
3/3 - 0s - loss: 2.0261 - accuracy: 0.7066 - val_loss: 2.0894 - val_accuracy: 0.0000e+00 - 72ms/epoch - 24ms/step
Epoch 4/50
3/3 - 0s - loss: 2.0010 - accuracy: 0.7101 - val_loss: 2.0919 - val_accuracy: 0.0000e+00 - 49ms/epoch - 16ms/step
Epoch 5/50
3/3 - 0s - loss: 1.9748 - accuracy: 0.7101 - val_loss: 2.0950 - val_accuracy: 0.0000e+00 - 52ms/epoch - 17ms/step
Epoch 6/50
3/3 - 0s - loss: 1.9477 - accuracy: 0.7101 - val_loss: 2.0987 - val_accuracy: 0.0000e+00 - 55ms/epoch - 18ms/step


In [ ]:
model.evaluate(x_test_pad, y_test)

20/20 [==============================] - 0s 7ms/step - loss: 2.0649 - accuracy: 0.4453


[2.064850091934204, 0.4453125]

In [ ]:
comment = input("comment:")
seq = tok.texts_to_sequences([comment])
pad = pad_sequences(seq, maxlen=62)
model.predict(pad)

comment:服用 adarone 宜 避免 或 小心 服用 葡萄柚 汁 請 二 服用


array([[0.12486216, 0.12369704, 0.12641554, 0.12534428, 0.12598531,
        0.12293393, 0.12220719, 0.1285546 ]], dtype=float32)

In [ ]:
###另外做語意分析模型
layers = [
    # 3001種 * 100 -> 300100
    Embedding(3001, 100, mask_zero=True),
    GlobalAveragePooling1D()
]
partial = Sequential(layers)
partial.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 100)         300100    
                                                                 
 global_average_pooling1d_2   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
Total params: 300,100
Trainable params: 300,100
Non-trainable params: 0
_________________________________________________________________


In [ ]:
w = model.layers[0].get_weights()
partial.layers[0].set_weights(w)

In [ ]:
from scipy.spatial.distance import cosine
word1 = "\u91AB\u751F\u6709\u958B\u7ACB\u4E0D\u5C11\u5E6B\u52A9\u5FAA\u74B0\u7684\u85E5\u7269\uFF0C\u518D\u8ACB\u4F60\u5011\u6CE8\u610F\uFF0C\u60A3\u8005\u6709\u6C92\u6709\u51FA\u73FE\u7600\u8840\u7684\u73FE\u8C61\u3002"#@param {type:"string"}
word2 = "Benzbromarone\u9808\u5B9A\u671F\u8FFD\u8E64\u809D\u80FD(\u60A3\u8005GOT\u6578\u503C73)\uFF0C\u82E5\u75DB\u98A8\u63A7\u5236\u826F\u597D\u5EFA\u8B70\u8207Febuxostat\u64C7\u4E00\u4F7F\u7528(\u672A\u898B\u5C3F\u9178\u6AA2\u9A57\u503C)\u65E9\u4E0A\u9700\u7559\u610F\u4F4E\u8840\u58D3\u60C5\u6CC1\u907F\u514D\u8DCC\u5012\u7559\u610F\u60A3\u8005\u5FC3\u8DF3\u901F\u7387(Quetiapine\u3001Levofloxacin\u3001Famotidine\u5408\u4F75\u4F7F\u7528)"#@param {type:"string"}
words = np.array([word1, word2])
seq = tok.texts_to_sequences(words)
pad = pad_sequences(seq)
result = partial.predict(pad)
# cos距離: 越接近1越相似
1 - cosine(result[0], result[1])

nan